<a href="https://colab.research.google.com/github/kamalahmadov474/LLM-based-NLP/blob/main/Retrieval_Augmented_Generation_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain transformers datasets faiss-cpu sentence-transformers rank_bm25
!pip install PyPDF2 pdfplumber gdown
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [ ]:
from google.colab import files
from PyPDF2 import PdfReader

# Upload your PDF
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

# Extract text from PDF pages
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

pdf_text = extract_text_from_pdf(pdf_path)
print(f"Extracted {len(pdf_text)} characters from PDF.")


Saving IAC 2024 Manuscript.pdf to IAC 2024 Manuscript.pdf
Extracted 17282 characters from PDF.


In [ ]:
from langchain.schema import Document

chunk_size = 500  # characters per chunk
chunks = [pdf_text[i:i+chunk_size] for i in range(0, len(pdf_text), chunk_size)]
documents = [Document(page_content=chunk, metadata={"source": "PDF Document"}) for chunk in chunks]

print(f"Created {len(documents)} document chunks.")


Created 35 document chunks.


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import pickle

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(documents, embedding_model)

# Save FAISS index for reuse
corpus_path = "faiss_index.pkl"
with open(corpus_path, "wb") as f:
    pickle.dump(vector_store, f)

print("FAISS vector store created and saved.")


/tmp/ipython-input-955456068.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

FAISS vector store created and saved.


In [ ]:
with open(corpus_path, "rb") as f:
    vector_store = pickle.load(f)

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})
print("Retriever created from FAISS index.")


Retriever created from FAISS index.


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from langchain.llms import HuggingFacePipeline

model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

llm_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=100)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

print(f"Loaded LLM model: {model_name}")


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cuda:0


Loaded LLM model: google/flan-t5-small


/tmp/ipython-input-970610076.py:9: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

prompt_template = """
Use the following retrieved documents to answer the question.
Context:
{context}
Question:
{question}
Answer:
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=False,
    chain_type_kwargs={"prompt": prompt}
)

print("RetrievalQA chain initialized.")


RetrievalQA chain initialized.


In [ ]:
queries = [
    "What is the PDF about?",
    "Summarize the main points of the document.",
    "Explain the background or context of this document.",
    "What problem does this document address?",
    "What methods or approaches are described?",
    "Give a brief overview of the introduction section."
]

for q in queries:
    result = qa_chain.run(q)
    print(f"Query: {q}")
    print(f"Answer: {result}\n")

Query: What is the PDF about?
Answer: The advent of artificial intelligence in space activities: New legal challenges. Space Policy 55, 101408. doi:10.1016/j.spacepol.2020.101 408 difficult.Additionally,the improving nature of AI technologies demonstrate s uncertainties regarding responsibility and blameworthiness in the event of accidents or damages. In order to resolve these issues,authorized frameworks have to adapt to the dynamics of AI -powered space communication.There must be evident guidelines to describe the

Query: Summarize the main points of the document.
Answer: A recent study achieved 96% accuracy using decision trees to distinguish between multi -path and ion communication and gives out fine - grained specification of numerous application areas based on AI that can resolve these impediments.A recent study achieved 96% accuracy using decision trees to distinguish between multi -path and ion communication and gives out fine - grained specification of numerous application a